**주피터 노트북 사용자:**
* 노트북에서 실행하실 분들은 `argparse` 삭제하시고 `batch_size`, `lr` 부분을 직접 함수에 인자로 넣으시면 되겠습니다.
* 코드 중간 `if __name__ == '__main__':` 은 파이썬에서 C/C++의 메인함수에 해당하는 부분입니다. 노트북에서는 다르게 작동하실 껍니다. 따라서 해당 라인을 삭제하고 함수만 실행시키시면 될겁니다.
* 저는 Multi-GPU를 사용하기 때문에 Multi-GPU가 불필요하신 분들은 해당 라인을 삭제하시고, Automatic Mixed Precision은 파이토치 1.5 버전부터 네이티브로 만들어진 기능입니다. 1.5 이하를 사용하시는 분들은 파이토치를 버전업하거나 해당 라인을 수정해서 사용하시기 바랍니다. 


**추가적인 팁:**
* 앙상블, 이미지 어그멘테이션, 스케줄러, 모형 변경 및 커스텀, 커스텀 로스 펑션 등을 순서대로 적용해보면서 자기만의 베이스라인과의 성능 차이를 기록하고 비교해보시기 바랍니다.
* `seed`를 고정하지 않았습니다. 병렬처리, 개발환경 등 모든 부분에 따라 다르기 때문에 캐글과 같이 비슷한 환경을 제공하거나 데이터셋이 너무 구려 가중치 초기화를 `seed`로 극복할 경우가 아닌 이상, 잘 학습된 모형의 경우 오차가 그리 크지 않을 겁니다.
* **중요: 모형을 커스텀할 때 꼭 레이어 초기화 방법을 설정해주시기 바랍니다. 성능 차이가 매우 큽니다.**
* **중요: 사용하실 모형과 딥러닝 프레임워크에 따라 인풋 데이터 형식이 다릅니다. 이미지 경우 채널 순서부터 정규화 방법까지 성능 차이가 매우 크므로 공식 문서를 꼭 읽고 따르시길 바랍니다. `torchvision.models`는 아래에 설명해놨습니다.**
* **중요: 코드공유에 공유된 Dacon.Jin님의 TTA를 적용해보시기 바랍니다.**

**참고: 코딩 스타일은 `<PEP8>`을 지킵니다. 파이썬 인덴트는 공백 4칸인데, 데이콘에 노트북 업로드 시 공백이 2칸으로 변경되네요. 4칸으로 수정해서 사용하시기 바랍니다.**

In [1]:
import os
import argparse
from typing import Tuple, Sequence, Callable
import csv
import cv2
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import KFold

import torch
import torch.optim as optim
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from torchvision import transforms
from torchvision.models import resnet101
import random

In [2]:
!nvidia-smi
print(torch.cuda.is_available())

Sat Feb 13 02:42:54 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 306...  Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   32C    P5    17W / 220W |    187MiB /  7982MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

In [4]:
seed_everything(42)

## 1. Out-of-Fold 전략을 위한 함수 정의

In [5]:
def split_dataset(path: os.PathLike) -> None:
    df = pd.read_csv(path)
    kfold = KFold(n_splits=5)
    for fold, (train, valid) in enumerate(kfold.split(df, df.index)):
        df.loc[valid, 'kfold'] = int(fold)

    df.to_csv('/home/gwonil/datasets/mnist_multilabel/split_kfold.csv', index=False)

## 2. 커스텀 데이터셋 정의 

In [6]:
class MnistDataset(Dataset):
    def __init__(
        self,
        dir: os.PathLike,
        image_ids: os.PathLike,
        transforms: Sequence[Callable]
    ) -> None:
        self.dir = dir
        self.transforms = transforms

        self.labels = {}
        with open(image_ids, 'r') as f:
            reader = csv.reader(f)
            next(reader)
            for row in reader:
                self.labels[int(row[0])] = list(map(int, row[1:]))

        self.image_ids = list(self.labels.keys())

    def __len__(self) -> int:
        return len(self.image_ids)

    def __getitem__(self, index: int) -> Tuple[Tensor]:
        image_id = self.image_ids[index]
        image = Image.open(
            os.path.join(self.dir, f'{str(image_id).zfill(5)}.png')).convert('RGB')
        target = np.array(self.labels.get(image_id)).astype(np.float32)

        if self.transforms is not None:
            image = self.transforms(image)

        return image, target

## 3. 이미지 어그멘테이션 정의

`torchvision.models`의 모형을 이용하여 transfer learning을 적용하고 싶으실 때 파이토치 공식 지원 `pillow`를 사용하지 않고, `opencv`를 사용하실려면:
* 채널 순서는 RGB
* `uint8` 0~255 값을 [0, 1] 값으로 정규화
* 정규화된 값을 ImageNet의 mean, std로 다시 정규화

어그멘테이션 적용 시, 해당 데이터셋의 특징을 잘 파악하고 적용하셔야 합니다. `albumentations`은 `ndarray`로 받습니다. `pillow`를 넘파이로 변경하시기 바랍니다.
본 대회의 데이터셋은 1채널이라 3채널로 변경해서 사용하시면 되겠습니다. 

In [7]:
transforms_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

transforms_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

## 4. 모형 정의 

In [8]:
class MnistModel(nn.Module):
    def __init__(self, num_classes: int = 26) -> None:
        super().__init__()
        self.resnet = resnet101(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)

        #nn.init.xavier_normal_(self.classifier.weight)

    def forward(self, x: Tensor) -> Tensor:
        x = self.resnet(x)
        x = self.classifier(x)

        return x

## 5. 학습

In [11]:
batch_size = 32
lr = 1e-3

def train(fold: int, verbose: int = 30) -> None:
    #split_dataset('/home/gwonil/datasets/mnist_multilabel/dirty_mnist_2nd_answer.csv')
    #df = pd.read_csv('/home/gwonil/datasets/mnist_multilabel/split_kfold.csv')
    #df_train = df[df['kfold'] != fold].reset_index(drop=True)
    #df_valid = df[df['kfold'] == fold].reset_index(drop=True)

    #df_train.drop(['kfold'], axis=1).to_csv(f'/home/gwonil/datasets/mnist_multilabel/train-kfold-{fold}.csv', index=False)
    #df_valid.drop(['kfold'], axis=1).to_csv(f'/home/gwonil/datasets/mnist_multilabel/valid-kfold-{fold}.csv', index=False)

    trainset = MnistDataset('/home/gwonil/datasets/mnist_multilabel/dirty_mnist_2nd', f'/home/gwonil/datasets/mnist_multilabel/train-kfold-{fold}.csv', transforms_train)
    train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)

    validset = MnistDataset('/home/gwonil/datasets/mnist_multilabel/dirty_mnist_2nd', f'/home/gwonil/datasets/mnist_multilabel/valid-kfold-{fold}.csv', transforms_test)
    valid_loader = DataLoader(validset, batch_size=32, shuffle=False, num_workers=4)

    num_epochs = 80
    device = 'cuda'
    scaler = GradScaler()

    model = MnistModel().to(device)
    #model.load_state_dict(torch.load('resnet101-f0-last.pth'))

    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MultiLabelSoftMarginLoss()
    
    best_acc = 0.0

    for epoch in range(num_epochs):
        model.train()
        for i, (images, targets) in enumerate(train_loader):
            optimizer.zero_grad()

            images = images.to(device)
            targets = targets.to(device)

            with autocast():
                outputs = model(images)
                loss = criterion(outputs, targets)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            if (i+1) % verbose == 0:
                outputs = outputs > 0.0
                acc = (outputs == targets).float().mean()
                print(f'Fold {fold} | Epoch {epoch}[{i+1:4d}/{len(train_loader)}] | L: {loss.item():.7f} | A: {acc.item():.7f}')

        model.eval()
        valid_acc = 0.0
        valid_loss = 0.0
        for i, (images, targets) in enumerate(valid_loader):
            images = images.to(device)
            targets = targets.to(device)

            with autocast():
                outputs = model(images)
                loss = criterion(outputs, targets)

            valid_loss += loss.item()
            outputs = outputs > 0.0
            valid_acc += (outputs == targets).float().mean()

        print(f'Fold {fold} | Epoch {epoch}[{len(valid_loader)}] | L: {valid_loss/(i+1):.7f} | A: {valid_acc/(i+1):.7f}\n')

        if valid_acc/(i+1) >= best_acc:
            torch.save(model.state_dict(), f'resnet101-f{fold}-best.pth')
            best_acc = valid_acc/(i+1)
        torch.save(model.state_dict(), f'resnet101-f{fold}-last.pth')
        
    print("best_acc: ", best_acc)

In [10]:
train(0)
train(1)
train(2)
train(3)
train(4)

Fold 0 | Epoch 0[  30/1250] | L: 0.7251595 | A: 0.5252404
Fold 0 | Epoch 0[  60/1250] | L: 0.6945106 | A: 0.5192308
Fold 0 | Epoch 0[  90/1250] | L: 0.6907578 | A: 0.5456731
Fold 0 | Epoch 0[ 120/1250] | L: 0.6930255 | A: 0.5132211
Fold 0 | Epoch 0[ 150/1250] | L: 0.6918434 | A: 0.5324519
Fold 0 | Epoch 0[ 180/1250] | L: 0.6888452 | A: 0.5420673
Fold 0 | Epoch 0[ 210/1250] | L: 0.6940880 | A: 0.5096154
Fold 0 | Epoch 0[ 240/1250] | L: 0.6929025 | A: 0.5348558
Fold 0 | Epoch 0[ 270/1250] | L: 0.6884880 | A: 0.5444711
Fold 0 | Epoch 0[ 300/1250] | L: 0.6931267 | A: 0.5240385
Fold 0 | Epoch 0[ 330/1250] | L: 0.6905042 | A: 0.5396635
Fold 0 | Epoch 0[ 360/1250] | L: 0.6901134 | A: 0.5372596
Fold 0 | Epoch 0[ 390/1250] | L: 0.6869604 | A: 0.5564904
Fold 0 | Epoch 0[ 420/1250] | L: 0.6893980 | A: 0.5288461
Fold 0 | Epoch 0[ 450/1250] | L: 0.6872994 | A: 0.5480769
Fold 0 | Epoch 0[ 480/1250] | L: 0.6907853 | A: 0.5420673
Fold 0 | Epoch 0[ 510/1250] | L: 0.6879373 | A: 0.5528846
Fold 0 | Epoch

Fold 0 | Epoch 3[ 510/1250] | L: 0.6873125 | A: 0.5480769
Fold 0 | Epoch 3[ 540/1250] | L: 0.6881884 | A: 0.5384616
Fold 0 | Epoch 3[ 570/1250] | L: 0.6877911 | A: 0.5492789
Fold 0 | Epoch 3[ 600/1250] | L: 0.6833514 | A: 0.5540866
Fold 0 | Epoch 3[ 630/1250] | L: 0.6868093 | A: 0.5516827
Fold 0 | Epoch 3[ 660/1250] | L: 0.6865000 | A: 0.5504808
Fold 0 | Epoch 3[ 690/1250] | L: 0.6882920 | A: 0.5360577
Fold 0 | Epoch 3[ 720/1250] | L: 0.6875001 | A: 0.5480769
Fold 0 | Epoch 3[ 750/1250] | L: 0.6903971 | A: 0.5264423
Fold 0 | Epoch 3[ 780/1250] | L: 0.6861014 | A: 0.5468750
Fold 0 | Epoch 3[ 810/1250] | L: 0.6844996 | A: 0.5504808
Fold 0 | Epoch 3[ 840/1250] | L: 0.6881216 | A: 0.5456731
Fold 0 | Epoch 3[ 870/1250] | L: 0.6928118 | A: 0.5276443
Fold 0 | Epoch 3[ 900/1250] | L: 0.6890426 | A: 0.5492789
Fold 0 | Epoch 3[ 930/1250] | L: 0.6894863 | A: 0.5360577
Fold 0 | Epoch 3[ 960/1250] | L: 0.6820174 | A: 0.5721154
Fold 0 | Epoch 3[ 990/1250] | L: 0.6851114 | A: 0.5540866
Fold 0 | Epoch

Fold 0 | Epoch 6[ 990/1250] | L: 0.6648221 | A: 0.6033654
Fold 0 | Epoch 6[1020/1250] | L: 0.6745836 | A: 0.5781250
Fold 0 | Epoch 6[1050/1250] | L: 0.6610212 | A: 0.5793269
Fold 0 | Epoch 6[1080/1250] | L: 0.6557381 | A: 0.6274039
Fold 0 | Epoch 6[1110/1250] | L: 0.6470643 | A: 0.6165866
Fold 0 | Epoch 6[1140/1250] | L: 0.6600499 | A: 0.5841346
Fold 0 | Epoch 6[1170/1250] | L: 0.6623560 | A: 0.5673077
Fold 0 | Epoch 6[1200/1250] | L: 0.6665940 | A: 0.5877404
Fold 0 | Epoch 6[1230/1250] | L: 0.6530776 | A: 0.6105769
Fold 0 | Epoch 6[313] | L: 0.6683909 | A: 0.5832897

Fold 0 | Epoch 7[  30/1250] | L: 0.6640670 | A: 0.5937500
Fold 0 | Epoch 7[  60/1250] | L: 0.6759442 | A: 0.5721154
Fold 0 | Epoch 7[  90/1250] | L: 0.6593703 | A: 0.6057693
Fold 0 | Epoch 7[ 120/1250] | L: 0.6495974 | A: 0.6129808
Fold 0 | Epoch 7[ 150/1250] | L: 0.6663274 | A: 0.6057693
Fold 0 | Epoch 7[ 180/1250] | L: 0.6661981 | A: 0.5913462
Fold 0 | Epoch 7[ 210/1250] | L: 0.6603435 | A: 0.6009616
Fold 0 | Epoch 7[ 2

Fold 0 | Epoch 10[ 210/1250] | L: 0.6001760 | A: 0.6622596
Fold 0 | Epoch 10[ 240/1250] | L: 0.6050898 | A: 0.6502404
Fold 0 | Epoch 10[ 270/1250] | L: 0.6233156 | A: 0.6382212
Fold 0 | Epoch 10[ 300/1250] | L: 0.6088441 | A: 0.6538462
Fold 0 | Epoch 10[ 330/1250] | L: 0.6177127 | A: 0.6514423
Fold 0 | Epoch 10[ 360/1250] | L: 0.6211332 | A: 0.6514423
Fold 0 | Epoch 10[ 390/1250] | L: 0.6321597 | A: 0.6394231
Fold 0 | Epoch 10[ 420/1250] | L: 0.6021053 | A: 0.6670673
Fold 0 | Epoch 10[ 450/1250] | L: 0.6198287 | A: 0.6430289
Fold 0 | Epoch 10[ 480/1250] | L: 0.6120129 | A: 0.6502404
Fold 0 | Epoch 10[ 510/1250] | L: 0.6160981 | A: 0.6406250
Fold 0 | Epoch 10[ 540/1250] | L: 0.6244389 | A: 0.6201923
Fold 0 | Epoch 10[ 570/1250] | L: 0.6093998 | A: 0.6574519
Fold 0 | Epoch 10[ 600/1250] | L: 0.6100639 | A: 0.6814904
Fold 0 | Epoch 10[ 630/1250] | L: 0.6194437 | A: 0.6406250
Fold 0 | Epoch 10[ 660/1250] | L: 0.6005985 | A: 0.6358173
Fold 0 | Epoch 10[ 690/1250] | L: 0.6134596 | A: 0.65985

Fold 0 | Epoch 13[ 630/1250] | L: 0.5780321 | A: 0.6826923
Fold 0 | Epoch 13[ 660/1250] | L: 0.5592169 | A: 0.6947116
Fold 0 | Epoch 13[ 690/1250] | L: 0.5433460 | A: 0.7115385
Fold 0 | Epoch 13[ 720/1250] | L: 0.5790724 | A: 0.6766827
Fold 0 | Epoch 13[ 750/1250] | L: 0.5679716 | A: 0.7067308
Fold 0 | Epoch 13[ 780/1250] | L: 0.5451019 | A: 0.7175481
Fold 0 | Epoch 13[ 810/1250] | L: 0.5577382 | A: 0.7019231
Fold 0 | Epoch 13[ 840/1250] | L: 0.5648578 | A: 0.6862981
Fold 0 | Epoch 13[ 870/1250] | L: 0.5722445 | A: 0.6802885
Fold 0 | Epoch 13[ 900/1250] | L: 0.5787276 | A: 0.6730769
Fold 0 | Epoch 13[ 930/1250] | L: 0.5627223 | A: 0.6971154
Fold 0 | Epoch 13[ 960/1250] | L: 0.5705992 | A: 0.6802885
Fold 0 | Epoch 13[ 990/1250] | L: 0.5224683 | A: 0.7379808
Fold 0 | Epoch 13[1020/1250] | L: 0.5386786 | A: 0.7223558
Fold 0 | Epoch 13[1050/1250] | L: 0.5431343 | A: 0.7091346
Fold 0 | Epoch 13[1080/1250] | L: 0.5297164 | A: 0.7223558
Fold 0 | Epoch 13[1110/1250] | L: 0.5822070 | A: 0.68028

...

Fold 0 | Epoch 73[ 630/1250] | L: 0.0498699 | A: 0.9819712
Fold 0 | Epoch 73[ 660/1250] | L: 0.0345211 | A: 0.9891827
Fold 0 | Epoch 73[ 690/1250] | L: 0.0363880 | A: 0.9855770
Fold 0 | Epoch 73[ 720/1250] | L: 0.0488524 | A: 0.9855770
Fold 0 | Epoch 73[ 750/1250] | L: 0.0370564 | A: 0.9879808
Fold 0 | Epoch 73[ 780/1250] | L: 0.0739398 | A: 0.9711539
Fold 0 | Epoch 73[ 810/1250] | L: 0.0565582 | A: 0.9843751
Fold 0 | Epoch 73[ 840/1250] | L: 0.0452288 | A: 0.9843751
Fold 0 | Epoch 73[ 870/1250] | L: 0.0542299 | A: 0.9831731
Fold 0 | Epoch 73[ 900/1250] | L: 0.0443511 | A: 0.9867789
Fold 0 | Epoch 73[ 930/1250] | L: 0.0307269 | A: 0.9891827
Fold 0 | Epoch 73[ 960/1250] | L: 0.0628152 | A: 0.9747596
Fold 0 | Epoch 73[ 990/1250] | L: 0.0542180 | A: 0.9771635
Fold 0 | Epoch 73[1020/1250] | L: 0.0306364 | A: 0.9903846
Fold 0 | Epoch 73[1050/1250] | L: 0.0530247 | A: 0.9807693
Fold 0 | Epoch 73[1080/1250] | L: 0.0452498 | A: 0.9819712
Fold 0 | Epoch 73[1110/1250] | L: 0.0454178 | A: 0.98317

Fold 0 | Epoch 76[1050/1250] | L: 0.0426530 | A: 0.9867789
Fold 0 | Epoch 76[1080/1250] | L: 0.0353111 | A: 0.9831731
Fold 0 | Epoch 76[1110/1250] | L: 0.0667610 | A: 0.9771635
Fold 0 | Epoch 76[1140/1250] | L: 0.0380923 | A: 0.9855770
Fold 0 | Epoch 76[1170/1250] | L: 0.0514706 | A: 0.9831731
Fold 0 | Epoch 76[1200/1250] | L: 0.0485309 | A: 0.9771635
Fold 0 | Epoch 76[1230/1250] | L: 0.0430823 | A: 0.9843751
Fold 0 | Epoch 76[313] | L: 0.5730351 | A: 0.8750194

Fold 0 | Epoch 77[  30/1250] | L: 0.0292221 | A: 0.9879808
Fold 0 | Epoch 77[  60/1250] | L: 0.0280442 | A: 0.9879808
Fold 0 | Epoch 77[  90/1250] | L: 0.0227322 | A: 0.9927885
Fold 0 | Epoch 77[ 120/1250] | L: 0.0188749 | A: 0.9915866
Fold 0 | Epoch 77[ 150/1250] | L: 0.0558661 | A: 0.9879808
Fold 0 | Epoch 77[ 180/1250] | L: 0.0530003 | A: 0.9795673
Fold 0 | Epoch 77[ 210/1250] | L: 0.0211715 | A: 0.9891827
Fold 0 | Epoch 77[ 240/1250] | L: 0.0345154 | A: 0.9903846
Fold 0 | Epoch 77[ 270/1250] | L: 0.0150162 | A: 0.9975962
Fo

Fold 1 | Epoch 0[ 210/1250] | L: 0.6915601 | A: 0.5288461
Fold 1 | Epoch 0[ 240/1250] | L: 0.6932825 | A: 0.5252404
Fold 1 | Epoch 0[ 270/1250] | L: 0.6904746 | A: 0.5252404
Fold 1 | Epoch 0[ 300/1250] | L: 0.6932769 | A: 0.5168269
Fold 1 | Epoch 0[ 330/1250] | L: 0.6932144 | A: 0.5216346
Fold 1 | Epoch 0[ 360/1250] | L: 0.6868721 | A: 0.5384616
Fold 1 | Epoch 0[ 390/1250] | L: 0.6889235 | A: 0.5480769
Fold 1 | Epoch 0[ 420/1250] | L: 0.6922666 | A: 0.5432693
Fold 1 | Epoch 0[ 450/1250] | L: 0.6843896 | A: 0.5468750
Fold 1 | Epoch 0[ 480/1250] | L: 0.6897718 | A: 0.5468750
Fold 1 | Epoch 0[ 510/1250] | L: 0.6883119 | A: 0.5456731
Fold 1 | Epoch 0[ 540/1250] | L: 0.6871629 | A: 0.5384616
Fold 1 | Epoch 0[ 570/1250] | L: 0.6882401 | A: 0.5588943
Fold 1 | Epoch 0[ 600/1250] | L: 0.6849542 | A: 0.5637019
Fold 1 | Epoch 0[ 630/1250] | L: 0.6896964 | A: 0.5300481
Fold 1 | Epoch 0[ 660/1250] | L: 0.6907475 | A: 0.5420673
Fold 1 | Epoch 0[ 690/1250] | L: 0.6901603 | A: 0.5312500
Fold 1 | Epoch

Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/gwonil/anaconda3/envs/pytorch/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/home/gwonil/anaconda3/envs/pytorch/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/gwonil/anaconda3/envs/pytorch/lib/python3.8/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/home/gwonil/anaconda3/envs/pytorch/lib/python3.8/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
  File "/home/gwonil/anaconda3/envs/pytorch/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/home/gwonil/anaconda3/envs/pytorch/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/gwonil/anaconda3/env

KeyboardInterrupt: 

In [9]:
#torch.cuda.empty_cache()

## 6. 테스트셋 제출 

In [10]:
def load_model(fold: int, epoch: int, device: torch.device = 'cuda') -> nn.Module:
    model = MnistModel().to(device)
    state_dict = 'resnet101-f' + str(fold) + '-best.pth'
    print(state_dict)

    model.load_state_dict(torch.load(state_dict))

    return model

In [11]:
def test(device: torch.device = 'cuda'):
    submit = pd.read_csv('/home/gwonil/datasets/mnist_multilabel/sample_submission.csv')

    model1 = load_model(0, 8)
    model2 = load_model(1, 8)
    model3 = load_model(2, 8)
    model4 = load_model(3, 8)
    model5 = load_model(4, 8)

    model1.eval()
    model2.eval()
    model3.eval()
    model4.eval()
    model5.eval()

    batch_size = test_loader.batch_size
    batch_index = 0
    for i, (images, targets) in enumerate(test_loader):
        images = images.to(device)
        targets = targets.to(device)

        outputs1 = model1(images)
        outputs2 = model2(images)
        outputs3 = model3(images)
        outputs4 = model4(images)
        outputs5 = model5(images)

        outputs = (outputs1 + outputs2 + outputs3 + outputs4 + outputs5) / 5

        outputs = outputs > 0.0
        batch_index = i * batch_size
        submit.iloc[batch_index:batch_index+batch_size, 1:] = \
            outputs.long().squeeze(0).detach().cpu().numpy()

    submit.to_csv('resnet101-e80-kfold.csv', index=False)

In [12]:
testset = MnistDataset('/home/gwonil/datasets/mnist_multilabel/test_dirty_mnist_2nd', '/home/gwonil/datasets/mnist_multilabel/sample_submission.csv', transforms_test)
test_loader = DataLoader(testset, batch_size=4, num_workers=2)
test()

resnet101-f0-8.pth
resnet101-f1-8.pth
resnet101-f2-8.pth
resnet101-f3-8.pth
resnet101-f4-8.pth
